# B站番剧评分数据分析和番剧推荐，用类型和年份进行用户评分预测

### 明确问题：

    明确要解决的问题：番剧推荐，番剧的产量 
    =>明确需要的数据：剧番id、标题、标签、长评数、短评数、年份、视频链接；用户id，剧番id，评分，评分时间

### 确定验证方案
  
     1）数据获取：爬虫，b站番剧
     2）数据处理
     3）数据展示和分析
     4）推荐算法+预测算法

### 设计预测模型
        
     1）新用户：基于内容的推荐
     2）老用户：基于协同过滤的推荐
     3）根据类型和时间进行新番的评分预测

### 实施流程：

    1、获取数据：爬虫
    2、了解数据：数据处理和展示
    3、构建模型：推荐，预测
 
#### 1 获取数据-爬虫：

    爬虫存在的问题：
        1）网络问题，会导致一些内容没有爬取到，而没存入csv中；
        2）可能有些特殊情况，比如评论时间的问题等，格式不一致或者其他情况，导致未爬取成功；
        这些情况导致的数据缺失问题，由一开始的基数进行弥补，即为了获取较多的数据，一开始选取的番剧数量也足够多。

    爬虫遇到的问题&解决方法：

| 出现的问题 | 猜测原因 | 解决方法 |
|: ------ |: ------ | :------ |
| 用request获取页面code无法获取真正的内容 | 页面是js动态加载的 | 借助selenium库，利用模拟浏览器模式打开网页获取内容<br/>```chrome_options = webdriver.ChromeOptions()<br/>#使用headless无界面浏览器模式```<br/>```chrome_options.add_argument('--headless') # 增加无界面选项```<br/>```chrome_options.add_argument('--disable-gpu') #如果不加这个选项，有时定位会出现问题```<br/>```#启动浏览器```<br/>```driver =webdriver.Chrome(options=chrome_options)``` <br/>```driver.get(url)```<br/>```content = driver.page_source```<br/>```soup = BeautifulSoup(content, 'lxml')```<br/>```driver.close()```<br/>|
| 突然访问失败，网页浏览也是 | ip被限制 | 设置随机等待时间，<br/>```rand_seconds = random.choice([1, 3]) + random.random()```<br/>  ```sleep(rand_seconds) ```|
| 重复爬取第一页内容 | 反爬机制 | 在更改url获取页面code的时候，	<br/>使用 driver.refresh()进行页面刷新 |
| url不变，下滑页面变化 | 动态加载 | ``` # 滑动动态加载页面，让浏览器执行简单的js代码```<br/> ```  js = "window.scrollTo(0,document.body.scrollHeight)"```<br/>``` driver.execute_script(js)``` |
| 长评论和短评论的分配 | -- | 按照比例分配，选取爬取评分的数量的基准是参考最少的评论总数 |

#### 2 了解数据-数据处理和展示

     数据处理：
| 爬虫过程 | 获得爬虫数据之后 | 
|:------ |:------ |
|  1\对重复爬取的link进行去重；<br/>2\对评论时间的处理，把只有月日、“昨天”，“x小时前”，<br/>“x分钟前”这样的非标准日期进行转化，变成年月日的日期形式；<br/>3\对于不存在评论的番剧已经进行跳过处理，不会爬取； | 1\对于还有可能的存在是数据重复进行处理<br/>2\数据标准化（对于番剧信息里面的'year'进行处理；<br/>提取评分和番剧信息里面的年，月，以便画图分析） |
    
    数据展示：
        统计：一共有多少番剧，一共有多少用户进行评分，
        柱状图：随着时间评分平均值的分布，总数的分布，剧番数量的分布；按月份的分布；每个番的每年或者每月评分分布情况；

        
#### 3 构建模型
    推荐：
    预测：

# 导包

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
%matplotlib inline
%config InlineBackend.figure_format ='svg'#为了画图更美观

# 数据标准化

In [54]:
#导入电影信息文件
video = pd.read_csv('video_data2.csv')
video.shape

(350, 7)

## 检查和去重

In [55]:
# 去重
video_df=video.drop_duplicates()
video_df.shape

(350, 7)

In [56]:
video_df.head()

,v_id,title,genres,year,long_comm,short_comm,detail_link
0,BV1Lb4113744,鬼灭之刃,"漫画改,战斗,热血,声控,",2019年4月7日,571,117950,http://www.bilibili.com/bangumi/media/md22718131/
1,BV1as411p7ae,工作细胞,"搞笑,战斗,日常,声控,",2018年7月8日,236,20754,http://www.bilibili.com/bangumi/media/md102392/
2,BV1pV411o7yD,辉夜大小姐想让我告白？～天才们的恋爱头脑战～,"搞笑,校园,恋爱,漫画改,",2020年4月12日,190,19169,http://www.bilibili.com/bangumi/media/md28228367/
3,BV11x411q7iu,欢迎来到实力至上主义的教室,"小说改,校园,",2017年7月13日,310,21943,http://www.bilibili.com/bangumi/media/md6339/
4,BV1ox411Q7V7,埃罗芒阿老师,"小说改,搞笑,恋爱,",2017年4月9日,152,15227,http://www.bilibili.com/bangumi/media/md5997/


In [57]:
rating = pd.read_csv('rating_data.csv')
rating.shape

(69690, 4)

In [58]:
# 去重
rating_df = rating_df.drop_duplicates()
rating_df.shape

(69657, 4)

In [59]:
rating_df.head()

,user_id_name,v_id,rating,rating_time
0,Canbuno,BV1Lb4113744,5,2019-12-29
1,永远爱食蜂,BV1Lb4113744,5,2020-05-01
2,叶子上的脉,BV1Lb4113744,5,2020-05-01
3,yourshoulds,BV1Lb4113744,5,2020-05-07
4,江小白_vlog,BV1Lb4113744,5,2020-05-06


## 番剧信息里面的'year'进行处理

In [60]:
video_df['year']

0       2019年4月7日
1       2018年7月8日
2      2020年4月12日
3      2017年7月13日
4       2017年4月9日
          ...    
345     2018年4月5日
346    2014年10月4日
347     2009年4月2日
348    2007年7月26日
349    2016年10月4日
Name: year, Length: 350, dtype: object

In [61]:
video_df['time']=video_df['year'].apply(lambda x: x.replace('年','-').replace('月','-').replace('日',''))
video_df['time']

0       2019-4-7
1       2018-7-8
2      2020-4-12
3      2017-7-13
4       2017-4-9
         ...    
345     2018-4-5
346    2014-10-4
347     2009-4-2
348    2007-7-26
349    2016-10-4
Name: time, Length: 350, dtype: object

In [62]:
video_df['v_year']=video_df['time'].apply(lambda x: x.split('-')[0])
video_df['v_month']=video_df['time'].apply(lambda x: x.split('-')[1])

In [65]:
video_df.head(2)

,v_id,title,genres,year,long_comm,short_comm,detail_link,time,v_year,v_month,rating_count
0,BV1Lb4113744,鬼灭之刃,"漫画改,战斗,热血,声控,",2019年4月7日,571,117950,http://www.bilibili.com/bangumi/media/md22718131/,2019-4-7,2019,4,118521
1,BV1as411p7ae,工作细胞,"搞笑,战斗,日常,声控,",2018年7月8日,236,20754,http://www.bilibili.com/bangumi/media/md102392/,2018-7-8,2018,7,20990


In [79]:
# 计算每个被评论的次数
video_df['rating_count'] = video_df.long_comm+video_df.short_comm
video_df.head(2)

,v_id,title,genres,year,long_comm,short_comm,detail_link,time,v_year,v_month,rating_count
0,BV1Lb4113744,鬼灭之刃,"漫画改,战斗,热血,声控,",2019年4月7日,571,117950,http://www.bilibili.com/bangumi/media/md22718131/,2019-4-7,2019,4,118521
1,BV1as411p7ae,工作细胞,"搞笑,战斗,日常,声控,",2018年7月8日,236,20754,http://www.bilibili.com/bangumi/media/md102392/,2018-7-8,2018,7,20990


In [81]:
# 取需要的属性列
new_video_df = video_df.iloc[:,[0,1,2,8,9,10]]
new_video_df.head(2)

,v_id,title,genres,v_year,v_month,rating_count
0,BV1Lb4113744,鬼灭之刃,"漫画改,战斗,热血,声控,",2019,4,118521
1,BV1as411p7ae,工作细胞,"搞笑,战斗,日常,声控,",2018,7,20990


In [69]:
# 获取每个的平均评分
rating_means = pd.DataFrame(rating_df.groupby('v_id').mean()['rating'])
rating_means.columns = ['rating_mean']
rating_means.head()

,rating_mean
v_id,
BV114411F7ac,4.656522
BV11W411B7fD,4.982609
BV11t411s776,4.965217
BV11t411s7Ay,4.952174
BV11x411q7iu,4.795652


In [85]:
# 合并数据
video_total = pd.merge(new_video_df, rating_means, on='v_id')
# video_total = video_total.sort_values(by="rating_count", ascending=False)
video_total.head(2)

,v_id,title,genres,v_year,v_month,rating_count,rating_mean
0,BV1Lb4113744,鬼灭之刃,"漫画改,战斗,热血,声控,",2019,4,118521,4.969565
1,BV1as411p7ae,工作细胞,"搞笑,战斗,日常,声控,",2018,7,20990,4.952174
